# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  
from sklearn.utils import shuffle 
import math
import scipy
import socket
import pickle

# Initialize Client-Server Commuication

In [ ]:
# Receive data until fully received (For Huge data => chunk by chunk)
def receive_all(socket, length):
    data = b''
    while len(data) < length:
        packet = socket.recv(length - len(data))
        if not packet:
            return None
        data += packet
    return data

# Send data in chunks(For Huge data => chunk by chunk)
def send_all(socket, data):
    data_pickle = pickle.dumps(data)
    data_size = len(data_pickle)
    socket.sendall(data_size.to_bytes(4, 'big'))  # Send data size first

    sent = 0
    while sent < data_size:
        chunk = data_pickle[sent:sent+4096]  # Send in chunks
        socket.sendall(chunk)
        sent += len(chunk)

# Initialize client socket
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_address = ('localhost', 1111)
client_socket.connect(server_address)
print('Client socket initialized')

# Get Server Initial Parameters

In [ ]:
# Receive initial params from server
initial_parameters_size = int.from_bytes(client_socket.recv(4), 'big')  # Receive data size first
initial_parameters_data = receive_all(client_socket, initial_parameters_size)
initial_parameters = pickle.loads(initial_parameters_data)
print('Received initial params from server')

# Load, Shuffle and Normalize Local Data

In [ ]:
# Load data according to server defined data-mode
data_mode = initial_parameters["data_mode"]
data = pd.read_excel(f'./client-datasets/{data_mode}/client-8.xlsx').values 

# Shuffle data
data = shuffle(data, random_state=42)

num_data = data.shape[0]
num_col = data.shape[1] 
num_class = 10 # fashion-mnist

# Normalize input data 
for ii in range(num_col-1): 
    data[:, ii] = ((data[:, ii] + 20) / 200)   # after test many  shift and scale value for data, with this values NN train better
    
percent_train = 0.7 # 70% Train 30% Test
num_train = round(num_data * percent_train)
num_test = num_data - num_train

# Convert labels to one-hot encoding (necessary for multi-class classification) 
y_one_hot_train = np.zeros((num_train, num_class))
y_one_hot_train[np.arange(num_train), data[:num_train, num_col-1].astype(int)] = 1

y_one_hot_test = np.zeros((num_test, num_class))
y_one_hot_test[np.arange(num_test), data[num_train:, num_col-1].astype(int)] = 1

# NN Structure

In [ ]:
n0 = data.shape[1]-1
n1 = 128
n2 = num_class    

# Set Parameters

In [ ]:
server_rounds = initial_parameters["server_rounds"] 
client_epochs = initial_parameters["client_epochs"] 
batch_size = initial_parameters["batch_size"]
lambda_reg = initial_parameters["lambda_reg"]

# optimization params setting
optimizer = initial_parameters["optimizer"]
optimization_params = initial_parameters["optimization_params"] 

# initialize mse_test according to server_rounds (after each aggregation proccess evaluate global-model with local test-data)
mse_test = np.zeros((server_rounds, len(optimization_params))) 

# Activation Functions

In [ ]:
ACTIVATION_FUNC = 'relu'
leaky_relu_alpha = 0.01
def activation_function(x,fun_name=ACTIVATION_FUNC):
    if(fun_name == 'relu'): 
        return np.maximum(0, x)
    elif(fun_name == 'logsig'): 
        return  1 /( 1 + (math.e)**(-1 * x))
    elif(fun_name == 'tansig'):
        return 2/(1+ (math.e)**(-2*x))-1
    elif(fun_name == 'leaky_relu'): 
        return np.where(x > 0, x, leaky_relu_alpha * x) 

def activation_function_derivative(x,fun_name=ACTIVATION_FUNC):
    if(fun_name == 'relu'): 
        return np.where(x > 0, 1, 0)
    elif(fun_name == 'logsig'): 
        logsig_x = activation_function(x)
        return logsig_x * (1 - logsig_x)
    elif(fun_name == 'tansig'):
        tansig_x = activation_function(x)
        return 1 - tansig_x**2
    elif(fun_name == 'leaky_relu'):
        return np.where(x > 0, 1, leaky_relu_alpha)

# Softmax Function

In [ ]:
def softmax(z, axis=None):
    exp_z = np.exp(z - np.max(z, axis=axis, keepdims=True))
    softmax_z = exp_z / np.sum(exp_z, axis=axis, keepdims=True)
    return softmax_z


# FedSGD - Simple Minibatch Gradient Descent

In [ ]:
def train_with_sgd(data_shuffled, y_one_hot_shuffled,  learning_rate, decay, t, w1, w2):
        error_data_train = np.zeros(num_train)
        output_data_train = np.zeros(num_train)
        
        for i in range(0, num_train, batch_size):
            # ******************************* feed-forward ******************************
            batch_data = data_shuffled[i:i+batch_size].T 
            batch_labels = y_one_hot_shuffled[i:i+batch_size] 
            
            net1 = w1 @ batch_data
            o1 = activation_function(net1)
            net2 = w2 @ o1
            o2 = net2
            z = softmax(o2,axis=0)
            output_data_train[i:i+batch_size] = np.argmax(z, axis=0)
            
            # ****************************** Backpropagation for each mini-batch ****************************** 
            # cross entropy error
            output_layer_error = np.mean(-(batch_labels.T * np.log(z, where=z>0)), axis=1)
            
            w2_old = w2
            # dw2 = dE/dz*dz/do2*do2/dnet2*dnet2/dw2 = (z - target) * fprim_net2 * o1
            dw2 = (z - batch_labels.T) @ o1.T / batch_size
            w2 = w2 - (learning_rate/(1+decay*t)) * dw2 - lambda_reg * w2

            fprim_net1 = activation_function_derivative(net1)
            # dw1 = dE/z*dz/do2*do2/dnet2*dnet2/do1*do1/dnet1*dnet1/dw1 = (z - target) * fprim_net2 * w2 * fprim_net1 * input_data
            dw1 = fprim_net1 * (w2_old.T @ (z - batch_labels.T)) @ batch_data.T / batch_size
            w1 = w1 - (learning_rate/(1+decay*t)) * dw1 - lambda_reg * w1

            error_data_train[i:i+batch_size] = output_layer_error.mean(axis=0)
        
        return {"w1": w1 ,"w2": w2,"output_data_train": output_data_train, "error_data_train": error_data_train}
    

# FedSGD + Nesterov momentum

In [ ]:
def train_with_nesterov_momentum(data_shuffled, y_one_hot_shuffled, learning_rate, decay, t, beta, w1, w2):
    error_data_train = np.zeros(num_train)
    output_data_train = np.zeros(num_train)
    vw1 = np.zeros((n1, n0))
    vw2 = np.zeros((n2, n1))

    for i in range(0, num_train, batch_size):
        # ******************************* feed-forward ******************************
        batch_data = data_shuffled[i:i+batch_size].T
        batch_labels = y_one_hot_shuffled[i:i+batch_size]

        # Nesterov Momentum update for w2
        w2_tilde = w2 - beta * vw2
        net1 = w1 @ batch_data
        o1 = activation_function(net1)
        net2 = w2_tilde @ o1
        o2 = net2
        z = softmax(o2, axis=0) 
        output_data_train[i:i+batch_size] = np.argmax(z, axis=0)
        
        # ****************************** Backpropagation for each mini-batch ****************************** 
        # cross entropy error
        output_layer_error = np.mean(-(batch_labels.T * np.log(z, where=z > 0)), axis=1)
        
        # dw2* = dE/dz*dz/do2*do2/dnet2*dnet2/dw2 = (z - target) * fprim_net2 * o1
        dw2_start = (z - batch_labels.T) @ o1.T / batch_size
        vw2 = beta * vw2 + (1 - beta) * dw2_start
        w2 = w2_tilde - (learning_rate / (1 + decay * t)) * vw2 - lambda_reg * w2

        # Nesterov Momentum update for w1
        fprim_net1 = activation_function_derivative(net1)
        
        # dw1* = dE/z*dz/do2*do2/dnet2*dnet2/do1*do1/dnet1*dnet1/dw1 = (z - target) * fprim_net2 * w2 * fprim_net1 * input_data
        dw1_star = fprim_net1 * (w2_tilde.T @ (z - batch_labels.T)) @ batch_data.T / batch_size
        vw1 = beta * vw1 + (1 - beta) * dw1_star
        w1 = w1 - (learning_rate / (1 + decay * t)) * vw1 - lambda_reg * w1

        error_data_train[i:i+batch_size] = output_layer_error.mean(axis=0)

    return {"w1": w1, "w2": w2, "output_data_train": output_data_train, "error_data_train": error_data_train}


# FedSGD + AdaGrad

In [ ]:
def train_with_adagrad(data_shuffled, y_one_hot_shuffled, learning_rate, decay, t, epsilon, w1, w2):
    error_data_train = np.zeros(num_train)
    output_data_train = np.zeros(num_train)
    sw1 = np.zeros((n1, n0))
    sw2 = np.zeros((n2, n1))

    for i in range(0, num_train, batch_size):
        # ******************************* feed-forward ******************************
        batch_data = data_shuffled[i:i+batch_size].T
        batch_labels = y_one_hot_shuffled[i:i+batch_size]

        net1 = w1 @ batch_data
        o1 = activation_function(net1)
        net2 = w2 @ o1
        o2 = net2
        z = softmax(o2, axis=0)
        output_data_train[i:i+batch_size] = np.argmax(z, axis=0)
        
        # ****************************** Backpropagation for each mini-batch ****************************** 
        output_layer_error = np.mean(-(batch_labels.T * np.log(z, where=z > 0)), axis=1)
        
        w2_old = w2
        # dw2 = dE/dz*dz/do2*do2/dnet2*dnet2/dw2 = (z - target) * fprim_net2 * o1
        dw2 = (z - batch_labels.T) @ o1.T / batch_size
        sw2 = sw2 + dw2 ** 2
        w2 = w2 - ((learning_rate / (1 + decay * t)) / np.sqrt(sw2 + epsilon)) * dw2 - lambda_reg * w2
        
        fprim_net1 = activation_function_derivative(net1)
        # dw1 = dE/z*dz/do2*do2/dnet2*dnet2/do1*do1/dnet1*dnet1/dw1 = (z - target) * fprim_net2 * w2 * fprim_net1 * input_data
        dw1 = fprim_net1 * (w2_old.T @ (z - batch_labels.T)) @ batch_data.T / batch_size
        sw1 = sw1 + dw1 ** 2
        w1 = w1 - ((learning_rate / (1 + decay * t)) / np.sqrt(sw1 + epsilon)) * dw1 - lambda_reg * w1

        error_data_train[i:i+batch_size] = output_layer_error.mean(axis=0)

    return {"w1": w1, "w2": w2, "output_data_train": output_data_train, "error_data_train": error_data_train}


# FedSGD + RMSProp

In [ ]:
def train_with_rmsprop(data_shuffled, y_one_hot_shuffled, learning_rate, decay, t, epsilon, beta, w1, w2):
    error_data_train = np.zeros(num_train)
    output_data_train = np.zeros(num_train)
    sw1 = np.zeros((n1, n0))
    sw2 = np.zeros((n2, n1))

    for i in range(0, num_train, batch_size):
        # ******************************* feed-forward ******************************
        batch_data = data_shuffled[i:i+batch_size].T
        batch_labels = y_one_hot_shuffled[i:i+batch_size]
 
        net1 = w1 @ batch_data
        o1 = activation_function(net1)
        net2 = w2 @ o1
        o2 = net2
        z = softmax(o2, axis=0)
        output_data_train[i:i+batch_size] = np.argmax(z, axis=0)
        
        # ****************************** Backpropagation for each mini-batch ****************************** 
        output_layer_error = np.mean(-(batch_labels.T * np.log(z, where=z > 0)), axis=1)
        
        w2_old = w2
        # dw2 = dE/dz*dz/do2*do2/dnet2*dnet2/dw2 = (z - target) * fprim_net2 * o1
        dw2 = (z - batch_labels.T) @ o1.T / batch_size
        sw2 = beta * sw2 +(1-beta) * (dw2 ** 2)
        w2 = w2 - ((learning_rate / (1 + decay * t)) / np.sqrt(sw2 + epsilon)) * dw2 - lambda_reg * w2
        
        fprim_net1 = activation_function_derivative(net1)
        # dw1 = dE/z*dz/do2*do2/dnet2*dnet2/do1*do1/dnet1*dnet1/dw1 = (z - target) * fprim_net2 * w2 * fprim_net1 * input_data
        dw1 = fprim_net1 * (w2_old.T @ (z - batch_labels.T)) @ batch_data.T / batch_size
        sw1 = beta * sw1 +(1-beta) * (dw1 ** 2)
        w1 = w1 - ((learning_rate / (1 + decay * t)) / np.sqrt(sw1 + epsilon)) * dw1 - lambda_reg * w1

        error_data_train[i:i+batch_size] = output_layer_error.mean(axis=0)

    return {"w1": w1, "w2": w2, "output_data_train": output_data_train, "error_data_train": error_data_train}


# FedSGD + Adam

In [ ]:
def train_with_adam(data_shuffled, y_one_hot_shuffled, learning_rate, decay, t, epsilon, beta1, beta2, w1, w2):
    error_data_train = np.zeros(num_train)
    output_data_train = np.zeros(num_train)
    sw1 = np.zeros((n1, n0))
    sw2 = np.zeros((n2, n1)) 
    
    vw1 = np.zeros((n1, n0))
    vw2 = np.zeros((n2, n1))
    
    vw1_hat = np.zeros((n1, n0))
    vw2_hat = np.zeros((n2, n1))
    
    sw1_hat = np.zeros((n1, n0))
    sw2_hat = np.zeros((n2, n1))

    for i in range(0, num_train, batch_size):
        # ******************************* feed-forward ******************************
        batch_data = data_shuffled[i:i+batch_size].T
        batch_labels = y_one_hot_shuffled[i:i+batch_size]
 
        net1 = w1 @ batch_data
        o1 = activation_function(net1)
        net2 = w2 @ o1
        o2 = net2
        z = softmax(o2, axis=0)
        output_data_train[i:i+batch_size] = np.argmax(z, axis=0)
        
        # ****************************** Backpropagation for each mini-batch ******************************
        output_layer_error = np.mean(-(batch_labels.T * np.log(z, where=z > 0)), axis=1)
        
        w2_old = w2
        # dw2 = dE/dz*dz/do2*do2/dnet2*dnet2/dw2 = (z - target) * fprim_net2 * o1
        dw2 = (z - batch_labels.T) @ o1.T / batch_size
        
        vw2 = beta1 * vw2 + (1 - beta1) * dw2
        vw2_hat = vw2/(1-beta1**t)
        
        sw2 = beta2 * sw2 +(1-beta2) * (dw2 ** 2)
        sw2_hat = sw2/(1-beta2**t) 
        
        w2 = w2 - ((learning_rate / (1 + decay * t)) / (np.sqrt(sw2_hat)+epsilon))*(beta1 * vw2_hat+((1-beta1)/(1-beta1**t))*dw2) - lambda_reg * w2

        
        fprim_net1 = activation_function_derivative(net1)
        # dw1 = dE/z*dz/do2*do2/dnet2*dnet2/do1*do1/dnet1*dnet1/dw1 = (z - target) * fprim_net2 * w2 * fprim_net1 * input_data
        dw1 = fprim_net1 * (w2_old.T @ (z - batch_labels.T)) @ batch_data.T / batch_size
        
        vw1 = beta1 * vw1 + (1 - beta1) * dw1
        vw1_hat = vw1/(1-beta1**t)
        
        sw1 = beta2 * sw1 +(1-beta2) * (dw1 ** 2)
        sw1_hat = sw1/(1-beta2**t)
        
        w1 = w1 - ((learning_rate / (1 + decay * t)) / (np.sqrt(sw1_hat)+epsilon))*(beta1 * vw1_hat+((1-beta1)/(1-beta1**t))*dw1) - lambda_reg * w1

        error_data_train[i:i+batch_size] = output_layer_error.mean(axis=0)

    return {"w1": w1, "w2": w2, "output_data_train": output_data_train, "error_data_train": error_data_train}


# Local Train Section

In [ ]:
def train_local_model(initials_parameters): 
    
    weights = initials_parameters["weights"]
    mse_train = np.zeros((client_epochs, len(optimization_params))) 
    output_data_trains = np.zeros((len(optimization_params), num_train))
    train_accuracy = np.zeros(len(optimization_params))
    np.random.seed(42) # for same suffled_indices for all experiments
    
    for t in range(client_epochs):

        shuffled_indices = np.random.permutation(num_train)
        data_shuffled = data[:num_train,:num_col - 1][shuffled_indices]
        y_one_hot_shuffled = y_one_hot_train[shuffled_indices]
        
        for index in range(len(optimization_params)):
            if(optimizer == 'sgd'):
                result = train_with_sgd(data_shuffled,
                                        y_one_hot_shuffled,
                                        optimization_params[index]["learning_rate"],
                                        optimization_params[index]["decay"],
                                        t,
                                        weights[index][0],
                                        weights[index][1]) # {w1,w2,output_data_train,error_data_train}
            elif(optimizer == 'nesterov_momentum'):
                result = train_with_nesterov_momentum(
                                        data_shuffled,
                                        y_one_hot_shuffled,
                                        optimization_params[index]["learning_rate"],
                                        optimization_params[index]["decay"],
                                        t,
                                        optimization_params[index]["momentum"],
                                        weights[index][0],
                                        weights[index][1]) # {w1,w2,output_data_train,error_data_train}
            elif(optimizer == 'adagrad'):
                result = train_with_adagrad(
                                        data_shuffled,
                                        y_one_hot_shuffled,
                                        optimization_params[index]["learning_rate"],
                                        optimization_params[index]["decay"],
                                        t,
                                        optimization_params[index]["epsilon"],
                                        weights[index][0],
                                        weights[index][1]) # {w1,w2,output_data_train,error_data_train}
            elif(optimizer == 'rmsprop'):
                result = train_with_rmsprop(
                                        data_shuffled,
                                        y_one_hot_shuffled,
                                        optimization_params[index]["learning_rate"],
                                        optimization_params[index]["decay"],
                                        t, 
                                        optimization_params[index]["epsilon"],
                                        optimization_params[index]["rho"],
                                        weights[index][0], weights[index][1]) # {w1,w2,output_data_train,error_data_train}
            elif(optimizer == 'adam'):
                result = train_with_adam(
                                        data_shuffled,
                                        y_one_hot_shuffled,
                                        optimization_params[index]["learning_rate"],
                                        optimization_params[index]["decay"],
                                        t+1,
                                        optimization_params[index]["epsilon"],
                                        optimization_params[index]["beta1"],
                                        optimization_params[index]["beta2"],
                                        weights[index][0],
                                        weights[index][1]) # {w1,w2,output_data_train,error_data_train}
            
            weights[index] = [result["w1"],result["w2"]]
            output_data_trains[index] = result["output_data_train"]
            mse_train[t][index] = np.mean(result["error_data_train"] ** 2)

        
        # Print training information
        print(f'Epoch: {t + 1}, Train MSE: {mse_train[t,:]}')

   
        for index in range(len(optimization_params)):
        
            print('***************************************************************************')
            print('optimization_params: ', optimization_params[index]["description"])
            
            # Plotting the training output
            plt.figure(figsize=(20, 8))
            plt.plot(data[:num_train, num_col-1][shuffled_indices], '-sr')  # Use shuffled indices for labels
            plt.plot(output_data_trains[index], '-*b')
            plt.xlabel('Train Data')
            plt.ylabel('Output')
            plt.title('Training Output')
            plt.show() 

            # plotting confusion matrix
            plot_confusion_matrix(output_data_trains[index], data[:num_train, num_col - 1][shuffled_indices], "Train Confusion Matrix")

            # Train accuracy
            train_accuracy[index] = np.mean(output_data_trains[index] == data[:num_train, num_col-1][shuffled_indices])
            print(f"Accuracy on the train set: {train_accuracy[index] * 100}%")  

    return {"client_id": "C8", "weights":weights, "train_mse":mse_train[-1,:], "train_accuracy": train_accuracy, "num_samples": num_data}
                                                                                                              

# Test Global Model(Aggregated Model) On Client Local Data

In [ ]:
def evaluate_aggregated_model(aggregated_parameters):   
    
    current_server_round = aggregated_parameters["current_server_round"]
    
    error_data_test = np.zeros((len(optimization_params), num_test))
    output_data_test = np.zeros((len(optimization_params), num_test))
    test_accuracy = np.zeros(len(optimization_params))
    
    for index in range(len(optimization_params)):
        for i in range(num_test):
            input_data = data[num_train + i, :num_col-1].reshape(-1,1) 
            w1 = aggregated_parameters['weights'][index][0]
            w2 = aggregated_parameters['weights'][index][1]
            net1 = w1 @ input_data 
            o1 = activation_function(net1)
            net2 = w2 @ o1 
            o2 = net2
            z = softmax(o2) 
            output_data_test[index][i] = np.argmax(z)
            
            error = -(y_one_hot_test[i:i+1] @ np.log10(z,where=z>0)).reshape(-1,1).flatten() 
            error_data_test[index][i] = error[0]
     

        mse_test[current_server_round][index] = np.mean(error_data_test[index] ** 2)
        
        print('optimization_params: ', optimization_params[index]["description"])
        
        # Plotting the test output
        plt.figure(figsize=(20, 8))
        plt.subplot(2, 2, 1)    
        plt.plot(data[num_train:, num_col-1], '-sr')
        plt.plot(output_data_test[index], '-*b')
        plt.xlabel('Test Data')
        plt.ylabel('Output')

        # Plotting the test MSE
        plt.subplot(2, 2, 2)
        plt.semilogy(np.arange(1, current_server_round + 1), mse_test[:current_server_round,index])
        plt.xlabel('Epoch')
        plt.ylabel('MSE Test')

        plt.tight_layout()
        plt.show()

        plot_confusion_matrix(output_data_test[index], data[num_train:, num_col - 1], "Test Confusion Matrix")

        print('current_server_round: {} \t'.format(current_server_round+1))
        print('MSE_Test: ' ,mse_test[current_server_round][index])


        test_accuracy[index] = np.mean(output_data_test[index] == data[num_train:,num_col-1]) 
        print(f"Accuracy on the test set: {test_accuracy[index] * 100}%") 
    
    return {"client_id":"C8" ,"test_mse": mse_test[current_server_round],'test_accuracy':test_accuracy, "num_samples": num_data}


# Plot Confusion Matrix

In [ ]:
def plot_confusion_matrix(predicted_classes, actual_classes, title): 
    
    # Create a confusion matrix-like matrix
    confusion_matrix = np.zeros((num_class, num_class))

    # Fill the confusion matrix
    for actual, predicted in zip(actual_classes, predicted_classes):
        confusion_matrix[actual.astype(int)][predicted.astype(int)] += 1 


    # Plot the confusion matrix
    plt.figure()
    plt.imshow(confusion_matrix, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(title)
    plt.colorbar()

    # Annotate the plot with numbers
    for i in range(num_class):
        for j in range(num_class):
            plt.text(j, i, str(int(confusion_matrix[i, j])), fontsize=12, ha='center', va='center')  # Corrected

    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.xticks(np.arange(num_class), np.arange(0, num_class))
    plt.yticks(np.arange(num_class), np.arange(0, num_class))

    plt.show()


# Client-Server Communication Section

In [ ]:
while True: 
    # get server signal => ready: start new learning process on client local data |  terminate: federated-learning process terminate by server
    signal = pickle.loads(client_socket.recv(4096))
    print('Server Signal: ', signal)
    
    if 'terminate' in signal:
        print("Received termination signal. Terminating client.")
        break 

    print('Start Local Training')

    # start train model with initial parameters already recieved from server on local Data
    data_to_send = train_local_model(initial_parameters)
    data_to_send['num_samples'] = num_data
    
    print('End Local Training')
    
    # send trained parameters back to server to aggregate
    send_all(client_socket, data_to_send)

    print('Sent Updated Params To Server')
    
    # receive aggregated model from server
    aggregated_parameters_size = int.from_bytes(client_socket.recv(4), 'big')  # Receive data size first
    aggregated_parameters_data = receive_all(client_socket, aggregated_parameters_size)
    aggregated_parameters = pickle.loads(aggregated_parameters_data)
    
    print('Received Aggregated Model From Server')
    
    print('Start Evaluating Aggregated Model With Local Test Data') 
        
    # evaluate aggregated model with test-data
    test_result = evaluate_aggregated_model(aggregated_parameters)
    
    print('End Evaluating Aggregated Model With Local Test Data')
    
    send_all(client_socket, test_result)

    print('Sent Evaluated Aggregated Model With Local Test Data Results To Server')
    
    # set updated-params to inital-params of next federated-learning local training process on client  
    initial_parameters = aggregated_parameters

    print("\n\033[1;m" + "*" * 125)   
    
# Close connection
client_socket.close()
